# Advection
Notebook to implement and test different methods on advection type PDEs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

from matplotlib import animation, rc
from IPython.display import HTML

import recon #for WENO import a copy of recon.py from pyclaw/limiters/weno

In [ ]:
def initial_advection(N,shape = 'delta',scale = 10*np.pi):
    """
    Method to generate test initila data for the advection equation
    
    Parmaters:
    N:     Number of points
    shape: Which testdata 
        'delta':   deltafunction
        'step':    the stepfunction
        'sin':     sine function
        'sin_cut': sine windowed
        'cos_cut': 1-cos windowed
    scale: Scale factor for 'sin' and 'sin_cut' in time
    """
    
    if shape == 'delta':
        u0 = np.zeros(N)
        u0[N//2] = 1
    elif shape == 'step':
        u0 = np.zeros(N)
        u0[:N//2] = 1
    elif shape == 'sin':
        x = np.linspace(0,1,N)
        u0 = np.sin(x*scale)+1
    elif shape == 'sin_cut':
        x = np.linspace(0,1,N)
        u0 = np.zeros(N)
        u0[(x*scale)<=np.pi] = np.sin(x*scale)[(x*scale)<=np.pi]
    elif shape == 'cos_cut':
        x = np.linspace(0,1,N)
        u0 = np.zeros(N)
        u0[(x*scale)<=(2*np.pi)] = 0.5*(1-np.cos(x*scale)[(x*scale)<=(2*np.pi)])
    elif shape == 'gauss':
        x = np.linspace(-0.5,0.5,N)
        u0 = np.exp(-(x**2/0.01))
        u0 = N*u0/np.sum(u0)
    else:
        print('wrong shape')             
    
    return u0
    



def plot_image(u,t):
    cmap=plt.get_cmap('plasma')
    extent=[0,t[-1],1,0]
    plt.imshow(u,cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    

def plot_at_time(u,T):
    n=np.argmin(np.abs(t-T))
    print(n,t[n])
    plt.plot(u[:,n])
    plt.title('t:'+str(t[n]))
    
    
def init():
    line.set_data([], [])
    time_text.set_text('')
    return (line,)

def animate(i,u,t):
    x = np.linspace(0,1,u.shape[0])
    y = u[:,i]
    line.set_data(x, y)
    time_template = 't = %.3f'
    time_text.set_text(time_template % (t[i]))
    return (line,)

def plot_animate(u,t,frames = 200):
    
    fig, ax = plt.subplots()

    ax.set_xlim(( 0, 1))
    ax.set_ylim((np.min(u), np.max(u)))
    
    global line
    global time_text
    
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    line, = ax.plot([], [], '-o', lw=2)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=range(0,u.shape[1],max(u.shape[1]//frames,1)), interval=200, 
                               blit=True,fargs = (u,t))
    return HTML(anim.to_jshtml())





In [ ]:
#Inplementing here the minmod interploation
def minmod(w,dx):
    """
    This function is an implementation for the minmod aproximation of a function
    
    it takes the vector of values w to compute the derivatives using the minmiod algorithm.
    The dw at the boundary are set to 0
    """
    
    
    forward = np.zeros_like(w)
    backward = np.zeros_like(w)
    
    forward[:-1] = (w[1:]-w[:-1])
    backward[1:] = (w[1:]-w[:-1])

    dw = (1/dx)*(np.sign(forward)+np.sign(backward))/2. * np.minimum(np.abs(forward),np.abs(backward))
    
    return dw

#How to encorporate boundaries?

def minmod_periodic(w,dx):
    """
    This function is an implementation for the minmod aproximation of a function
    
    it takes the vector of values w to compute the derivatives using the minmiod algorithm.
    for periodic boundaries
    """
    
    forward = np.zeros_like(w)
    backward = np.zeros_like(w)
    
    forward[:-1] = (w[1:]-w[:-1])
    forward[-1] = w[0]-w[-1]
    backward[1:] = (w[1:]-w[:-1])
    backward[0] = w[0]-w[-1]

    dw = (1/dx)*(np.sign(forward)+np.sign(backward))/2. * np.minimum(np.abs(forward),np.abs(backward))
    
    return dw


# method to test other types of limiters. Interface the same as for minmod. 
# The limiter can bes set using a global variable
limiter  = 'minmod'
def reconstruct(y,dx):

    
    forward  = np.zeros_like(y)
    backward = np.zeros_like(y)
    
    
    forward[:-1] = (y[1:]-y[:-1])
    backward[1:] = (y[1:]-y[:-1])
    theta = forward[backward!=0]/backward[backward!=0]
    
    
    
    
    if limiter == 'minmod':
        phi = (1+np.sign(theta))/2. * np.minimum(1,theta)
    elif limiter == 'vanleer':
        phi = (theta + np.abs(theta))/(1+np.abs(theta))
    elif limiter == 'MC':
        phi = np.maximum(0,np.minimum( (1.+theta)/2., np.minimum(2.,theta)))
    elif limiter == 'superbee':
        phi = np.maximum(0,np.maximum(np.minimum(1.,2*theta),np.minimum(2.,theta)))
    else:
        print('no known limiter')
        raise ValueError

    sigma = np.zeros_like(y)
    sigma[backward!=0] = phi * backward[backward!=0]
    return (1/dx)*sigma

In [ ]:
#Test for limiter:
limiter  = 'MC'
k=10
x=np.linspace(0,1,k)
y=np.sin(x*5)+1.
#y = (x>0.5)*1.
dx = 1/k

sigma = reconstruct(y,dx)

xx = np.linspace(0,1,1000)
diff = np.abs(x.reshape(1,-1) - xx.reshape(-1,1))
closest = np.argmin(diff,axis=1)
yy = y[closest] + sigma[closest]*(xx-x[closest])

plt.plot(xx,yy,'-k',lw=2)
plt.plot(x,y,'or',markersize=10,alpha=0.5)

In [ ]:
#test recon.weno
k=11
x=np.linspace(0,1,k)
#y=np.sin(x*5)+1.
y = (x>0.5)*1.
#y[x == 0.5] = 0.5
dx = 1/k

q = y.copy()
q.shape = (1,k)
ql,qr=recon.weno(5,q,epweno=1e-5)
ql.shape=(k,)
qr.shape=(k,)

xl = x-dx/2
xr = x+dx/2

xx = np.zeros(2*k)
yy = np.zeros(2*k)

xx[range(0,2*k,2)] = xl
xx[range(1,2*k,2)] = xr

yy[range(0,2*k,2)] = ql
yy[range(1,2*k,2)] = qr

plt.plot(xx,yy,'-k',lw=2)
plt.plot(x,y,'or',markersize=10,alpha=0.5)

# Linear advection

In [ ]:
#u0 = (x<0.5)*1.
#u0 = (x<0.5)+0.2
u0 = (x<0.5)+0.
#u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx




N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]


def f_upwind_advection(t,u):
    dx = 1/len(u)
    du = np.zeros_like(u)
    du[1:] = (u[1:]-u[:-1])
    du[0] = u[0]-u[-1]
    return -du/(dx)


def f_upwind_advection_so(t,u): ##secnd order aproxiamtion
    dx = 1/len(u)
    du = np.zeros_like(u)
    
    dw = reconstruct(u,dx)
    ur = u + 0.5*dx*dw
    
    du[1:] = 1/dx*(-ur[1:] + ur[:-1])
    return du

def f_upwind_advection_weno(t,u):
    dx = 1/len(u)
    du = np.zeros_like(u)
    
    u.shape =(1,len(u))
    ul,ur=recon.weno(5,u,epweno=1e-5)
    ur.shape=(len(du),)
    du[1:] = 1/dx*(-ur[1:] + ur[:-1])
    return du



CFL condition:

Advection coefficent $a$ = 1 

This leads to $$\frac{dx}{dt} \leq 1$$

In [ ]:
limiter = 'minmod'
N =100


methods = (ssp104,dp5,ck5)

solver_ssp104 = Solver(rkm = ssp104,
               dt = 1,
               t_final = 10,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_dp5 = Solver(rkm = dp5,
               dt = 1,
               t_final = 10,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_ck5 = Solver(rkm = ck5,
               dt = 1,
               t_final = 10,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})


u0 = initial_advection(100,'delta')

problem_upwind_advection = Problem(f=f_upwind_advection_so,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

dts = utils.findall_dt(problem_upwind_advection,(solver_ssp104,solver_dp5,solver_ck5),
                      dt_start =0.009,tol = 0.0001,Params = {})
#dts = utils.findall_dt(RK_variable_b,methods,f_upwind_advection_so,u0,dt_start=0.009,tol=0.0001,
#                   Params=dict(t_final=0.5,b_fixed=True,fallback = True,num_fallback= 4,
#                      LP_opts = {'reduce':True,'feastol':1e-10}))
print(dts)




In [ ]:
utils.plot_times(methods,dts,effective=False,title='Advection linear')

$\bullet$ simple upwinding

For the linear advection with the simple upwinding scheme we can see that the ssp104 ensures positifity for all stable timesteps. 
The dp5 and ck5 methods do not ensure positifity for the hole stable region. The regions with positifity can be extended by adapting the b while enforcing the Order Conditions for 4th Order.


$\bullet$ Second order approximations (with superbee)
For the second order approxiamtions we encounter negative values inside the stability region (with superbee)

ssp104: $dt_{feasible}$ inbetween $dt_{pos}$  and $dt_{stable}$. But the numeric $t_{stable}$ is greater than expected. The value is probably wrong.

dp5: $dt_{pos}$ and $dt_{feasible}$ very smal. This is stays the same when reducing the Order.

ck5: $dt_{pos} = dt_{feasible} \leq dt_{stable}$ Cannot be changed by reducing the Order.


$\bullet$ Second order approximations (with minmod)
Also possivble improvements

ssp104: The same baviour as for superbee. Inside the expected stability region the ssp104 solution is positive anyway.

dp5: $dt_{pos}$ is very small. $dt_{feasible}$ is nearly as big as $dt_{stable}$

ck5: $dt_{pos}$ is quite big. $dt_{feasible}$ is between $dt_{pos}$ and $dt_{stable}$




$\bullet$ weno5

When using the weno5 reconstruction we encounter issuses with positifity for all timesteps at all methods. These cannot be fixed by adapting the bs





In [ ]:
limiter = 'minmod'

u0 = initial_advection(100,'cos_cut')
#u0 = np.random.rand(100)

dx = 1/100
dt = dx*4.5
#dt = 0.015 #works for fallback = 1 wyh dp5
#dt = 0.017
#dt = 0.015 #works for fallback = 1 wyh ck5

solver = Solver(rkm = ssp104,
               dt = dt,
               t_final = 0.5,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

problem_upwind_advection = Problem(f=f_upwind_advection_so,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

status,t,u,b = RK_integrate(solver=solver,problem=problem_upwind_advection,verbose=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)


In [ ]:
plot_image(u,t)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
np.min(u)

In [ ]:
plot_at_time(u,0.01399999)
plt.ylim(-0.00001,0.00001)

In [ ]:
plot_animate(u,t,frames=len(t))

Possible good settings:

limiter = minmod, rkm = dp5, dt = 0.014, u0 = cos_cut

# Variable Coefficent Advection

$u_t + (a(x,t)u)_x = 0$

$u_t = - (a(x,t)u)_x$

$\frac{\mathrm d}{\mathrm d x} \big( a(x,t)u(x,t) \big) = \frac{\mathrm d}{\mathrm d x} \big( u(x,t) \big) a(x,t) =\frac{\mathrm d}{\mathrm d x} \big( a(x,t) \big) u(x,t)$


Hom. Dirichelt Boundary at $x=0$ open Boundary on $x=1$

In [ ]:


def f_var_coeff_adv(t,u):
    
    #Old implementation, does not enshure conservation
    #A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    #a = lambda x,t: np.cos(20*x + 45*t)**2
    #a_ = lambda x,t: 2*np.cos(20*x + 45*t)*-np.sin(20*x + 45*t)*20
    #return np.diag(a(x,t))@A_upwind_advection@u +  a_(x,t)*u
    
    N = len(u)
    dx = 1/N
    
    x = np.linspace(0,1,N)
    
    A_upwind_advection = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    a = lambda x,t: np.cos(20*x + 45*t)**2
    return A_upwind_advection@(a(x,t)*u)


def f_adv_step(t,u):
    N = len(u)
    dx = 1/N
    x = np.linspace(0,1,N)
    w = u*((np.abs(x-0.5)<0.2)*0.5+0.5)
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-w[1:] + w[:-1])
    return du

def f_adv_step_so(t,u):
    N = len(u)
    dx = 1/N
    x = np.linspace(0,1,N)
    
    dw = reconstruct(u,dx)
    ur = u + 0.5*dx*dw
    w = ur*((np.abs(x-0.5)<0.2)*0.5+0.5)
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-w[1:] + w[:-1])
    return du

def f_adv_step_weno(t,u):
    du = np.zeros_like(u)
    u.shape=(1,len(u))
    ql,qr=recon.weno(5,u)
    qr.shape=(len(u),)
    
    x = np.linspace(0,1,N)
    w = qr*((np.abs(x-0.5)<0.2)*0.5+0.5)
    
    
    du[1:] = 1/dx*(-w[1:] + w[:-1])
    return du

In [ ]:
N =100
limiter = 'minmod'

u0 = initial_advection(100,'cos_cut')

methods = (ssp104,dp5,ck5)

solver_ssp104 = Solver(rkm = ssp104,
               dt = 1,
               t_final = 2,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_dp5 = Solver(rkm = dp5,
               dt = 1,
               t_final = 2,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_ck5 = Solver(rkm = ck5,
               dt = 1,
               t_final = 2,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})


u0 = initial_advection(100,'delta')

problem_advection_step = Problem(f=f_adv_step_so,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

dts = utils.findall_dt(problem_advection_step,(solver_ssp104,solver_dp5,solver_ck5),
                      dt_start =0.009,tol = 0.0001,Params = {})

print(dts)

In [ ]:
utils.plot_times(methods,dts,effective=False,title='variable coefficent advection')

#### Variable Advection

For the variable advection case we can see that the ssp104 does not ensures positifity for all stable timesteps. The region with positife timesteps can be enlarged by adapting the b.

The dp5 and ck5 methods also do not ensure positifity for the hole stable region. When enforcing the Order Conditions for 4th Order $\Delta t_{feasible} = \Delta t_{pos}$. 
When reducing the Conditions to 3d Order $\Delta t_{feasible} > \Delta t_{pos}$. A possible explaination would be that there are not enough degrees of freeedom for the b to find a feasible b. When reducing the Order this increases the degrees of freedom for the the choice of the b. 
This explaination does stil not explain why the linear advection does not show this behavior.

#### Regions with different a

For the weno reconstruction and the minmod, superbee the behaviour is similar. 

ssp104: All stable timestepsare also positive 

dp5 and ck5: At the boundaries of the different regions there are oscilations occuring that reduce the positive stepsize. These oscilations can be suppressed by adapting the bs. This can cause distortions on other places. Maybe this is a interesting testcase for a local adaptation of the bs.


In [ ]:
limiter = 'minmod'
#dt = 0.09
#dt = 0.10799999999999998
#dt = dt -0.02
#dt = 0.001
#N = 500
#dt = 0.023625 +0.002
dt = 0.024
dt = 0.01*2
#ck5: 1.2 woorks good with 1.3 shows slight oscilations 1.15 very interesting without optim

solver = Solver(rkm = dp5,
               dt = dt,
               t_final = 2,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = np.inf,
               p = [4,3,2],
               theta = [1],
               solver = cp.ECOS,
               LP_opts = {})

N =100
u0 = initial_advection(N,'cos_cut')

problem_advection_step = Problem(f=f_adv_step_so,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)


status,t,u,b = RK_integrate(solver=solver,problem=problem_advection_step,verbose=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plot_image(u,t)
#plot_image(1.*(u<-0),t)
#plt.imshow(u,cmap=plt.get_cmap('seismic'),vmin=-5e-14, vmax=5e-14)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
plot_at_time(u,1.8)

In [ ]:
plot_animate(u,t,frames=300)

In [ ]:
np.min(u)

Possible Settings:

limiter = minmod, rkm = dp5, dt = 0.02, u0 = cos_cut

# Burgers Equation

$$u_t + (u^2)_x = 0$$

$$u_t = - (u^2)_x$$

Flux differencing upwind

$$u_i' = \frac{1}{\Delta x} (f(u_1)-f(u_{u-1}))$$

Hom. Dirichelt Boundary at $x=0$ open Boundary on $x=1$

In [ ]:
def f_burgers(t,u):
    N = len(u)
    dx = 1/N
    
    x = np.linspace(0,1,N)
    w = 0.5*u**2
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-w[1:] + w[:-1]) 
    return du


def f_burgers_dir(t,u):
    #burger equation with constant value at x = 0
    N = len(u)
    dx = 1/N
    
    ux0 = 1
    
    w = 0.5*u**2
    wx0 = 0.5*ux0**2
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-w[1:] + w[:-1]) 
    du[0] =  1/dx*(-w[0] + wx0)
    return du



In [ ]:
def f_burgers_so(t,u):
    N = len(u)
    dx = 1/N
    w = 0.5*u**2
    dw = reconstruct(w,dx)
    
    ww = w + 0.5*dx*dw
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-ww[1:] + ww[:-1])
    return du


def f_burgers_minmod_I(t,u):
    #burgers equation with minmod interpolation on flux function for geting the derivatives
    N = len(u)
    dx = 1/N
    w = 0.5*u**2
    dw = minmod(w,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    ww = w + 0.5*dx*dw
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-ww[1:] + ww[:-1])
    return du

def f_burgers_minmod(t,u):
    #burgers equation with minmod on u for geting the derivatives
    N = len(u)
    dx = 1/N

    q = minmod(u,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    uu = u + 0.5*dx*q
    ww = 0.5*uu**2
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-ww[1:] + ww[:-1])
    return du

def f_burgers_minmod_per_I(t,u):
    #burgers equation with minmod interpolation on flux function for geting the derivatives
    N = len(u)
    dx = 1/N
    w = 0.5*u**2
    dw = minmod_periodic(w,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    ww = w + 0.5*dx*dw
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*0.5*(-ww[1:] + ww[:-1])
    du[0] = 1/dx*0.5*(-w[0] + w[-1])
    return du

def f_burgers_minmod_per(t,u):
    #burgers equation with minmod interpolation of u for geting the derivatives
    N = len(u)
    dx = 1/N
    q = minmod_periodic(w,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    uu = u + 0.5*dx*q
    ww = 0.5*uu**2
    du = np.zeros_like(u)
    du[1:] = 1/dx*0.5*(-ww[1:] + ww[:-1])
    du[0] = 1/dx*0.5*(-w[0] + w[-1])
    return du

def f_burgers_minmod_dir(t,u):
    #burgers equation with minmod for geting the derivatives 
    #using u(0) = 1
    N = len(u)
    dx = 1/N
    u_ = np.zeros(len(u)+1)
    u_[1:] = u
    u_[0] = 1
    q = minmod(u_,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    uu = u_ + 0.5*dx*q
    ww = 0.5*uu**2
    du = np.zeros_like(uu)
    du[1:] = 1/dx*(-ww[1:] + ww[:-1])
    du[0] = 1/dx*(-ww[0] + ww[-1])
    return du[1:]

def f_burgers_minmod_dir_I(t,u):
    #burgers equation with minmod interpolation on flux function for geting the derivatives
    #using u(0) = 1
    N = len(u)
    dx = 1/N
    w = np.zeros(len(u)+1)
    w[1:] = 0.5*u**2
    w[0] = 0.5
    dw = minmod(w,dx)
    
    #We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    #because we know u >= 0 we know the fluxes are allways to the right
    #we approximate the values for the fluxes with extraploationg to the right uu(i-1/2)
    ww = w + 0.5*dx*dw
    
    du = np.zeros_like(w)
    du[1:] = 1/dx*(-ww[1:] + ww[:-1])
    du[0] = 1/dx*(-ww[0] + ww[-1])
    return du[1:]

In [ ]:
def f_burgers_weno(t,u):
    shape = u.shape
    u.shape=[1,len(u)]
    ql,qr=recon.weno(5,u,epweno=1e-2)
    qr.shape = shape
    wr = 0.5*qr**2
    
    du = np.zeros(shape)
    du[1:] = 1/dx*(-wr[1:] + wr[:-1])
    return du

def f_burgers_weno_dir(t,u):
    b = 5
    shape = u.shape
    u_=np.ones([1,len(u)+b])
    #u.shape=[len(u),1]
    u_[0,b:] = u
    ql,qr=recon.weno(5,u_,epweno=1e-6)
    qr.shape=(len(u)+b,)
    
    wr = 0.5*qr**2
    
    du = np.zeros(shape)
    du = 1/dx*(-wr[b:] + wr[b-1:-1])
    du.shape = shape
    return du



In [ ]:
N =100
limiter = 'superbee'

u0 = initial_advection(100,'step')
#u0 = np.zeros_like(u0)



methods = (ssp104,dp5,ck5)

solver_ssp104 = Solver(rkm = ssp104,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_dp5 = Solver(rkm = dp5,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_ck5 = Solver(rkm = ck5,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})


u0 = initial_advection(100,'delta')

problem_burgers = Problem(f=f_burgers,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

dts = utils.findall_dt(problem_burgers,(solver_ssp104,solver_dp5,solver_ck5),
                      dt_start =0.001,tol = 0.00005,Params = {})




#methods = (fe,ssp104,dp5,ck5,ssp3,merson4,pd8)
#methods = (ssp104,dp5,ck5)
#dts = utils.findall_dt(RK_variable_b,methods,f_burgers,u0,dt_start=0.001,tol=0.00005,
#                   Params=dict(t_final=1,b_fixed=True,fallback = True,num_fallback= 2,
#                      LP_opts = {'reduce':True,'feastol':1e-10}))
print(dts)




In [ ]:
utils.plot_times(methods,dts,effective=False,title='Burgers equation')

$\bullet $ Simple upwind discretisation

ssp104: For the Burgers equation the ssp104 method ensures positifity for all stable timesteps.

dp5: $dt_{pos}$ is slightly smaler than $dt_{feasible} = dt_{stable}$ but adapting the bs does not improve the magnitude of the negative values

ck5: $dt_{pos}=dt_{feasible} = dt_{stable}$

$\bullet$ Second order approxiamtion (with minmod)

ssp104: allways positive

dp5: $dt_{pos}$ is smaler. The negative values can be slightly reduced but are stil to big

ck5: $dt_{pos}=dt_{feasible} = dt_{stable}$

$\bullet$ Second order approxiamtion (with superbee)

ssp104: allways positive

dp5: $dt_{pos}=dt_{feasible} < dt_{stable}$ for Order 4, 
$dt_{feasible}$ increases with Order 3 but dapting the b does not improve the magnitude of negative values

ck5: $dt_{pos}=dt_{feasible} = dt_{stable}$ 


$\bullet$ weno 

Simmilar to above

In [ ]:
limiter = 'superbee'

N =100
dx = 1./N
dt = 2.*dx*0.5*1.2 #gives values >1 for 2.*dx*3.
print(dt)

solver = Solver(rkm = dp5,
               dt = dt,
               t_final = 1,
               b_fixed=True,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

u0 = initial_advection(N,'step')
problem_burgers = Problem(f=f_burgers_so,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)


#u0 = np.zeros(N)
#t,u,b,KK,status = RK_variable_b(dp5,dt,f_burgers_so,w0=u0,t_final=1,b_fixed=True,fallback = True,num_fallback=4,
#                      dumpK=True,maxval = None,return_status=True,LP_opts={'reduce':True},solver='scipy_sim')
#display(status)

status,t,u,b = RK_integrate(solver=solver,problem=problem_burgers,verbose=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plot_image(u,t)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
#plot_at_time(u,0.4)
plot_at_time(u,0.0)

In [ ]:
plot_animate(u,t,frames=300)

In [ ]:
#plot_image(((u>1)|(u<0))*1.,t)
plot_image((u<0)*1.,t)

In [ ]:
np.min(u)

Possible settings:
    
limiter = minmod, rkm = dp5, dt = 0.016, u0 = step

## Traffic equation

form Leveque 2002 p.204

$$ q_t +(qU(q))_x =0 $$

with

$$U(q)=u_{max}(1−q) for 0 \leq q \leq 1$$

## Further ideas to make a problem that breaks positifity

Trying to construct an example that has a hight advection coefficent when solution near 0:

Start with quasilinear formulation of equation

$$ u_t + f'(u) u_x = 0 $$

Contrary to the burgers equation i want a $f'(0)$ to be bigg and $max\{f'(u)\}$ to be smal on the uesd range to provide a bigger stability region. 
We choose 
$$ f'= \frac{1}{u+0.1} $$

We integrate $f'$ and get

$$ f = \log(0.1 + q) + C$$

We want the solution to ensure positifity for the semidisretisation. To ensure this we set $f(0) = 0$ and get

$$ f = \log(0.1 +q) - \log(0.1) $$

The restriction on the staility region can be derived form $max\{f'(u)\}$. In our case this is $f'(0) = 10$. 
We get a CFL conditon of $\frac{dt}{dx} \leq 0.1$


Needs some further developement

In [ ]:
def f_transport_per(t,u):
    N=len(u)
    v_max = 1
    w = v_max*u*(1-u) #fluxes at values 
    
    #take fluxes in between, decide based on direction of caracteristics
    ww = np.where(u[:-1]+u[1:]<=1,w[:-1],w[1:]) #fluxes inbetween
    wwq = np.where(u[0]+u[-1]<=1,w[-1],w[0])
    
    du = np.zeros_like(u)
    du[1:-1] = 1/dx*(ww[:-1]-ww[1:])
    du[0] = 1/dx*(wwq-ww[0])
    du[-1] = 1/dx*(ww[-1]-wwq)
    
    return du
    
    

def f_transport_minmod_per(t,u):
    #burgers equation with minmod for geting the derivatives
    N = len(u)
    dx = 1/N
    v_max = 1
    w = v_max*u*(1-u)
    dw = minmod_periodic(w,dx)
    
    
    """
    We want to calculate the flux at the borders betwwen the cells. We interploate w with the minmod method
    because we cann not calculate the direction of the flux a-priori we calculate it from both sides
    with the sign of the characteristic speed we u_max(1 − 2q) we determine which one to use
         for q<0.5 to the right -> use left value
         for q>0.5 to the left  -> use right value
    We use the average of two adjacent points to approxiamte the direction. 
    If it is eqactly half it is not well defined
    """
    
    wl = w + 0.5*dx*dw
    wr = w - 0.5*dx*dw
    
    ww = np.where(u[1:]+u[:-1]<=1,wl[:-1],wr[1:]) ##ww is used for the interiot fluxes
    wwp = np.where(u[0]+u[-1]<=1,wl[-1],wr[0])    #flux at x = 0 and x =1
    
    du = np.zeros_like(u)
    du[1:-1] = 1/dx*0.5*(-ww[1:] + ww[:-1])
    du[-1] = 1/dx*0.5*(-wwp + ww[-1])
    du[0] = 1/dx*0.5*(-ww[0] + wwp)
    return du


#def f_inv_burger_weno(t,u):
#    shape = u.shape
#    u.shape=[1,len(u)]
#    ql,qr=recon.weno(5,u)
#    qr.shape = shape
#    wr = qr-0.1*np.log(qr+0.1)+0.1*np.log(0.1) #use value form right because the flow is to the right
    
#    du = np.zeros(shape)
#    du[1:] = 1/dx*(-wr[1:] + wr[:-1])
#    return du

def f_inv_nonlin_burger(t,u):
    N = len(u)
    dx = 1/N
    
    wr = np.log(0.1 +u) - np.log(0.1)
    
    du = np.zeros_like(u)
    du[1:] = 1/dx*(-wr[1:] + wr[:-1])
    return du
    

In [ ]:
N =100

u0 = initial_advection(100,'gauss')

methods = (ssp104,dp5,ck5)

solver_ssp104 = Solver(rkm = ssp104,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_dp5 = Solver(rkm = dp5,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

solver_ck5 = Solver(rkm = ck5,
               dt = 1,
               t_final = 1,
               b_fixed=False,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})

problem_inv_nonlin = Problem(f=f_inv_nonlin_burger,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

dts = utils.findall_dt(problem_inv_nonlin,(solver_ssp104,solver_dp5,solver_ck5),
                      dt_start =0.001,tol = 0.00005,Params = {})


print(dts)

In [ ]:
utils.plot_times(methods,dts,effective=False,title='Transport equation')

In [ ]:
N =100
dx = 1./N
dt = 0.1*dx*3.


print(dt)
u0_ = initial_advection(N,'cos_cut')
s = 10
#u0[:s] = u0_[-s:];u0[s:] = u0_[:-s]
u0 = u0_

solver = Solver(rkm = ssp104,
               dt = dt,
               t_final = 4,
               b_fixed=True,
               tol_neg=1e-9,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = 'scipy_ip',
               LP_opts = {})


problem_inv_nonlin = Problem(f=f_inv_nonlin_burger,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)


status,t,u,b = RK_integrate(solver=solver,problem=problem_inv_nonlin,verbose=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plot_image(u,t)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
#plot_at_time(u,0.4)
plot_at_time(u,0.024) 

In [ ]:
plot_animate(u,t,frames=300) 

In [ ]:
plot_image((u<0)*1.,t)

In [ ]:
np.min(u)

In [ ]:
u[-1,-1]